In [110]:
import wmfdata as wmf
import pandas as pd
import numpy as np
from wmfdata import spark,hive
from datetime import datetime

In [133]:
file = "https://analytics.wikimedia.org/published/datasets/knowledge_gaps/content_gaps/csv/geography_wmf_region.csv"
geodata = pd.read_csv(file)

# filter for wikipedia projects
wikis=pd.read_csv('data/wikis')
geodata=geodata[geodata.wiki_db.isin(list(wikis['database_code']))]

In [134]:
# Since each snapshot contains patial data of the month it is published it in: we filter the data from previous month if the data was pulled too early. *Needs to be double checked.

# Convert 'time_bucket' to datetime for comparison
geodata['time_bucket'] = pd.to_datetime(geodata['time_bucket'])


# If today's day is before the 23rd of the month, filter out the previous month's data.  
if datetime.now().day < 23:
    prev_month = datetime.now().month - 1 if datetime.now().month > 1 else 12
    geodata = geodata[geodata['time_bucket'].dt.month != prev_month]

In [135]:
old_geo_data = pd.read_csv('data/updated_geo_data.csv') # load current geo_data table

old_geo_data['time'] = pd.to_datetime(old_geo_data['time'])

old_geo_data # view data

,time,Latin America & Caribbean total,Central & Eastern Europe & Central Asia total,"East, Southeast Asia, & Pacific total",North America total,Northern & Western Europe total,Middle East & North Africa total,Sub-Saharan Africa total,South Asia total,UNCLASSED total,...,Central & Eastern Europe & Central Asia yoy average (3m),"East, Southeast Asia, & Pacific yoy average (3m)",North America yoy average (3m),Northern & Western Europe yoy average (3m),Middle East & North Africa yoy average (3m),Sub-Saharan Africa yoy average (3m),South Asia yoy average (3m),UNCLASSED yoy average (3m),total,underrepresented
0,2001-01-01,0,0,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,2001-02-01,0,0,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,2001-03-01,0,0,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,2001-04-01,0,0,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,2001-05-01,0,0,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,2023-04-01,58862,236966,157340,176200,537194,35548,26686,41712,NaN,...,21159.0,13372.0,11542.0,31550.0,2848.0,2182.0,3435.0,NaN,89531.0,0.282360
268,2023-05-01,59063,238517,158246,176858,540325,35793,26930,42151,NaN,...,20295.0,13139.0,11289.0,34245.0,2885.0,2209.0,3548.0,NaN,91050.0,0.277002
269,2023-06-01,59231,239573,158980,177288,541987,35944,27102,42329,NaN,...,19441.0,12943.0,10536.0,36515.0,2888.0,2234.0,3616.0,NaN,91540.0,0.273629
270,2023-07-01,59538,242139,160736,178593,546113,36188,27710,42999,265.0,...,19282.0,13023.0,10030.0,37126.0,2888.0,2380.0,3822.0,NaN,91816.0,0.276401


In [136]:
# Group and pivot the geodata

# remove underrepresented column
if 'underrepresented' not in filtered_last_rows.columns:
    filtered_last_rows['underrepresented'] = 0

    
quality_totals = geodata.groupby(['time_bucket', 'category'])['standard_quality_count_value'].sum().reset_index()
quality_totals_pivot = quality_totals.pivot(index='time_bucket', columns='category', values='standard_quality_count_value').reset_index()

quality_totals_pivot['time'] = pd.to_datetime(quality_totals_pivot['time_bucket'])

cols_to_rename = {col: col + " total" for col in quality_totals_pivot.columns if col not in ['time_bucket', 'category', 'time']}
quality_totals_pivot = quality_totals_pivot.rename(columns=cols_to_rename)


quality_totals_pivot


category,time_bucket,Central & Eastern Europe & Central Asia total,"East, Southeast Asia, & Pacific total",Latin America & Caribbean total,Middle East & North Africa total,North America total,Northern & Western Europe total,South Asia total,Sub-Saharan Africa total,UNCLASSED total,time
0,2001-01-01,0,0,0,0,0,0,0,0,0,2001-01-01
1,2001-02-01,0,0,0,0,0,0,0,0,0,2001-02-01
2,2001-03-01,0,0,0,0,0,0,0,0,0,2001-03-01
3,2001-04-01,0,0,0,0,0,0,0,0,0,2001-04-01
4,2001-05-01,0,0,0,0,0,0,0,0,0,2001-05-01
...,...,...,...,...,...,...,...,...,...,...,...
245,2023-04-01,237364,157856,58867,35431,176454,538260,41826,26946,265,2023-04-01
246,2023-05-01,238928,158771,59072,35676,177115,541405,42264,27202,265,2023-05-01
247,2023-06-01,240510,159704,59321,35952,177772,543828,42665,27452,265,2023-06-01
248,2023-07-01,242139,160736,59538,36188,178593,546113,42999,27710,265,2023-07-01


In [137]:
# Identify the latest 'time' in old_geo_data

latest_time = old_geo_data['time'].max()

# Filter rows from total quality articles where 'time' is greater than latest_time from old_data to identify rows to be added
filtered_rows = quality_totals_pivot[quality_totals_pivot['time'] > latest_time]

# Filter columns of last_row based on columns in geo_data
old_geo_data = pd.concat([old_geo_data, filtered_rows], axis=0, ignore_index=True, sort=False)


old_geo_data # display data


,time,Latin America & Caribbean total,Central & Eastern Europe & Central Asia total,"East, Southeast Asia, & Pacific total",North America total,Northern & Western Europe total,Middle East & North Africa total,Sub-Saharan Africa total,South Asia total,UNCLASSED total,...,"East, Southeast Asia, & Pacific yoy average (3m)",North America yoy average (3m),Northern & Western Europe yoy average (3m),Middle East & North Africa yoy average (3m),Sub-Saharan Africa yoy average (3m),South Asia yoy average (3m),UNCLASSED yoy average (3m),total,underrepresented,time_bucket
0,2001-01-01,0,0,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaT
1,2001-02-01,0,0,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaT
2,2001-03-01,0,0,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaT
3,2001-04-01,0,0,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaT
4,2001-05-01,0,0,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,2023-04-01,58862,236966,157340,176200,537194,35548,26686,41712,NaN,...,13372.0,11542.0,31550.0,2848.0,2182.0,3435.0,NaN,89531.0,0.282360,NaT
268,2023-05-01,59063,238517,158246,176858,540325,35793,26930,42151,NaN,...,13139.0,11289.0,34245.0,2885.0,2209.0,3548.0,NaN,91050.0,0.277002,NaT
269,2023-06-01,59231,239573,158980,177288,541987,35944,27102,42329,NaN,...,12943.0,10536.0,36515.0,2888.0,2234.0,3616.0,NaN,91540.0,0.273629,NaT
270,2023-07-01,59538,242139,160736,178593,546113,36188,27710,42999,265.0,...,13023.0,10030.0,37126.0,2888.0,2380.0,3822.0,NaN,91816.0,0.276401,NaT


In [138]:
# check if new data was appended to the original csv file.

if filtered_rows.empty:
    print("No data has been changed or added to the original spreadsheet. Will not overwrite updated_geo_data.csv")
    print(old_geo_data)

else:
    # Compute the Year-over-Year difference
    yoy_difference = old_geo_data.drop(columns='time').diff(periods=12)

    # Compute the 3-month rolling average for the entire dataset
    rolling_avg_yoy = yoy_difference.rolling(window=3).mean()

    # Round every value to the nearest whole number
    rolling_avg_yoy = rolling_avg_yoy.round(0)

    # Add the 'time' column back
    rolling_avg_yoy['time'] = old_geo_data['time']
    
    # Make a list of all columns for every individual region.
    columns_to_sum = []
    
    # Get 3m rolling average (yoy)
    for column in rolling_avg_yoy.columns:
        if column != 'time':
            new_column_name = column.strip().replace(" total", "")
            columns_to_sum.append(new_column_name)
            print(new_column_name)  # Debugging line
            old_geo_data[f"{new_column_name}"] = rolling_avg_yoy[column]


    # Reorder columns to make 'time' the first column
    rolling_avg_yoy = rolling_avg_yoy[['time'] + [col for col in rolling_avg_yoy if col != 'time']]
    

    # Create 'total' column
    print(columns_to_sum)
    old_geo_data['total'] = old_geo_data[[col for col in columns_to_sum]].sum(axis=1)
    

    # Create 'underrepresented' column which contains the percentage for the metric.

    # List of underrepresented regions
    underrepresented_regions = [
        "East, Southeast Asia, & Pacific", 
        "Latin America & Caribbean", 
        "Middle East & North Africa", 
        "South Asia", 
        "Sub-Saharan Africa"
    ]

    # Compute the sum of underrepresented regions
    old_geo_data['underrepresented_sum'] = old_geo_data[underrepresented_regions].sum(axis=1)

    # Calculate the proportion of underrepresented regions to total
    old_geo_data['underrepresented'] = old_geo_data['underrepresented_sum'] / old_geo_data['total']

    # drop underrepresented sum
    old_geo_data.drop('underrepresented_sum', axis=1, inplace=True)

    # Save the updated geo_data to a CSV
    
    # rename columns
    old_geo_data = old_geo_data.drop(columns="time_bucket")
    rename_dict = {col: f"{col} yoy average (3m)" for col in columns_to_sum if col in old_geo_data.columns}
    old_geo_data = old_geo_data.rename(columns=rename_dict)
    
    # output
    print("updating csv file")
    old_geo_data.to_csv('data/updated_geo_data.csv', index=False)

No data has been changed or added to the original spreadsheet. Will not overwrite updated_geo_data.csv
          time  Latin America & Caribbean total  \
0   2001-01-01                                0   
1   2001-02-01                                0   
2   2001-03-01                                0   
3   2001-04-01                                0   
4   2001-05-01                                0   
..         ...                              ...   
267 2023-04-01                            58862   
268 2023-05-01                            59063   
269 2023-06-01                            59231   
270 2023-07-01                            59538   
271 2023-08-01                            59840   

     Central & Eastern Europe & Central Asia total  \
0                                                0   
1                                                0   
2                                                0   
3                                                0   
4             